<span style="font-family:Lucida Bright;">
<p style="margin-bottom:0.5cm"></p>
<center>
<font size="8"><b>Deep Learning, Fall 2021</b></font>
<p style="margin-bottom:0.6cm"></p>
<font size="3"><b>Final Project:</b></font>
<p style="margin-bottom:0.6cm"></p>
<font size="5"><b>Enhancing Voices for Better Speech Intelligibility</b></font>
<p style="margin-bottom:2cm"></p>
<font size="6"><b>Start</b></font>
</center>
<p style="margin-bottom:2cm"></p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#How-to-get-the-most-out-of-this-notebook" data-toc-modified-id="How-to-get-the-most-out-of-this-notebook-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>How to get the most out of this notebook</a></span></li><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Links-and-resources" data-toc-modified-id="Links-and-resources-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Links and resources</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#TIMIT-dataset" data-toc-modified-id="TIMIT-dataset-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>TIMIT dataset</a></span></li><li><span><a href="#Synthetic-speech-dataset" data-toc-modified-id="Synthetic-speech-dataset-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Synthetic speech dataset</a></span></li></ul></li></ul></li></ul></div>

# Introduction

Many people struggle to understand speech in challenging acoustic environments, such as noisy bar. Therefore, enhancing the intelligibility of noisy speech signals is one of the key challenges for any producer of modern communication devices.

The problem is often tackled by dividing a noisy speech signal into a number of frequency bands and attenuating the ones where the signal-to-noise ratio is insufficient. This approach, while effective in some situations, often leads to poor results, and sometimes even exacerbates the problem it is trying to solve as the constant activation and disactivation of some of the frequency bands in response to the fluctuations in speech and noise can create a very unnatural and disturbing sounds.

In this project, we will try a different approach and attempt to create a deep learning model that will produce an equalization curve that can be applied to the noisy speech signal in order to maximize its intelligibility. This will be done by running a clear speech signal through a model of human auditory processing of and searching for a combination of parameters that produce a frequency-gain curve that, when applied to the noisy signal, creates the output most similar to that of clear speech.

## Official project description

In the info-doc for the Deep Learning course, the project is described as follows:

> **Designing self-driving earbuds with [augmentedhearing.io](augmentedhearing.io) which enhance voices based on function correlated with speech intelligibility**
>
> As one in four adults struggle to understand speech in challenging acoustics we aim to train consumer earbuds to enhance voices through back propagation using DHASP model implemented using [PyTorch differentiation package](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571) validated on [TIMIT speech dataset](https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3) based on an objective function correlated with [HASPI speech intelligibility auditory processing model](https://www.sciencedirect.com/science/article/pii/S0167639320300431) available in Matlab.

## The approach

The time scope for the course is normed to 7 days of 9 hours, which amounts to 63 hours. The outcome should be documented in a report formatted as a [conference paper](https://drive.google.com/file/d/0BxJRy96AHCJxaUEwOFhwUExmX00/view?usp=sharing&resourcekey=0-RvwJqDVrZVijbkkifLWoYA), as well as a Jupyter notebook that ideally should recreate the main results of the report.

At the beginning of the project, the following resources were available:

1. speech data: a TIMIT dataset consisting of 10 sentences spoken by 630 speakers from 8 major dialect regions of the United States
2. an [article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571) describing a proposal for a differentiable objective function that can be used to train a neural network. The function is a measure of similarity of a [cepstrum](https://en.wikipedia.org/wiki/Cepstrum) of a given speech signal to the cepstrum of the target signal (usually clear, noisless speech)
3. Matlab code for calculating the [HASPI speech intelligibility index] that can be used to evaluate the results.

In turn, to create a our model, we need the following:

1. clear speech audio data to use as the target for model
2. the corresponding noisy speech audio data to train the model
3. a PyTorch implementation of the equalization filter that can be applied to the noisy speech signal. Our model will optimize the parameters of this filter to maximize speech intelligibility
4. a working PyTorch implementation of the objective function proposed in the [DHASP article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571).

Therefore, before we could start creating and tweaking our neural network, we needed to obtain the prerequisites 2 - 4, of which especially number 4: implementation of the objective function proposed in the [DHASP article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571) was, as I will demonstrate later, all but straightforward.

To obtain a fully functional model would require going through and understanding the project literature, obtaining the prerequisites, crating and optimizing the neural network, and documenting of the findings, which is a task that extends way beyond the 63-hour scope of this project. I have dedicated more than 3 times as much time, and concentrated my effort on obtaining a functioning implementation of the all the prerequisites necessary to build a neural network that can be used to optimize our objective. Judged by the results I will present in this notebook, I might have succeeded.

# How to get started

This notebook is a part of project. To be able to execute all of its content:

1. Download the project repository: `https://github.com/wojciechdk/deep-learning-2021-final-project.git` and run the notebook from the root.
2. Install the necessary packages specified in the file `[project_root]/requirements.txt`. It can be done in one hook by running the command `pip install -r requirements.txt` from the command prompt from the project root.
3. Download the [TIMIT dataset](https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3), unpack it, and place it so that the folders *DOC*, *TEST*, and *TRAIN* are placed in the folder `[project_root]/resources/data/TIMIT`.
4. Download the Sythetic Speech dataset created by Pawel Maciej Darulewski (for permission, please contact Pawel at s200123@student.dtu.dk). Place the data from the folder containing full length sentences in the folder `[project_root]/resources/data/synthetic_speech/full_length` and the data from the folder containing 5s segments in the folder `[project_root]/resources/data/synthetic_speech/cut_5_s`.

Furthermore, to fully enjoy the content, please take note of the following:

- The outputs of pre-executed cells may not be rendered properly unless the notebook is **Trusted**.
- To avoid accidental changes, most of the cells in this notebook are marked-as read only, and many are frozen (i.e. disabled from being run). To take advantage of these features, it is recommended to use the extension
[Freeze Cell](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/freeze/readme.html) which works with Jupyter Notebook.

## The organization of the project repository

### Functions and modules

To keep the code clean, all the functions created by the author in the course of this project are defined in the modules of the package `toolbox` residing the folder `[project_root]/toolbox`. Each module contains the functions belonging to the category indicated by the name of the module.

The following modules are used in the project:

- `initialization`: contains all the code that needs to be executed before anything else, such as imports of necessary packages, setting of options, definition of project paths, etc.
- `imports`: contains the imports of all the packages needed in this project.
- `configuration`: contains the code that defines the options regarding the appearance and interactivity of the Jupyter Notebook, Pandas, etc.
- `paths`: contains a class containing all the paths necessary to run this project.
- `data_loading`: contains the functions that help load the data into meaningful structures, such as the functions that load all the metadata about the TIMIT and Synthetic Speech datasets into respective Pandas dataframes.
- `dhasp`: contains the class containing a PyTorch implementation of the Differentiable Hearing Aid Speech Processing (DHASP) model described in this [article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571)
- `dsp`: contains the functions used for processing of signals and extracting their metrics.
- `plotting`: contains the functions that help plot the data in this project.
- `sound`: contains the functions make it easy to listen to the audio used in this project.
- `type_conversion`: contains the functions that facilitate the conversion between different data types, e.g. numpy and a torch.
- `general`: contains the functions that were not given a category of their own.

### Test scripts

All the scripts used for testing the code in produced in this project are placed in the folder: `[project_root]/tests`.

### Resources

The project's resources are placed in the folder `[project_root]/resources` and include:
- `cache`: the cached data, such as the metadata about the TIMIT and Synthetic Speech dataset.
- `data`: the audio files for the TIMIT and Synthetic Speech dataset.
- `matlab`: the Matlab code containing the functions used for calculation of the [HASPI speech intelligibility auditory processing model](https://www.sciencedirect.com/science/article/pii/S0167639320300431)
- `stoi_examples`: contains a Jupyter Notebook containing examples of how to use a Python package to calculate speech intelligibility metrics alternative to HASPI.


# Initialization

The entire initialization process, including:
 - imports of the necessary packages
 - configuration of the notebook and packages
 - imports of the toolbox functions

is defined in the file `[project_root]/toolbox/initialization.py`. Let's run it:

In [2]:
from toolbox.initialization import *

Everything is now initialized and the project paths are available in the variable `paths`. Let's view one path:

In [3]:
display(paths.data.timit.root)

WindowsPath('G:/My Drive/DTU/Kurser/Deep_Learning_02456/final_project/resources/data/TIMIT')

# Load information about the audio data

At the beginning of the project, the only audio data I had at my disposal was the [TIMIT dataset](https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3). This dataset consists of 10 sentences spoken by 630 speakers from 8 major dialect regions of the United States. However, the data contains only recordings containing clean speech, while our objective function which measures similarity between the [cepstral](https://en.wikipedia.org/wiki/Cepstrum) sequences for two audio signals, requires both clean and noisy versions of the same speech segment.

Instead of generating the noisy data, I was offered by Pawel Maciej Darulewski to use a set containing samples of synthetically generated speech in different acoustical situations, which he has created for a similar project. In the following, I will therefore use Pawel's audio data. I have, however, implemented functions that allow easy access to both the TIMIT and Pawel's Synthetic Speech datasets, which I will present in the following sections.



## TIMIT dataset

The metadata about the TIMIT dataset is loaded using the function `load_timit_data` defined in the module `[project_root]\toolbox\data_loading.py`. Once loaded, the data is saved in the project cache as a Pandas dataframe. Let's load it and show a couple of rows:

In [4]:
# Load the TIMIT meta data from the cache.
df_timit = pd.read_pickle(paths.cache.df_timit)

# Show the top 5 rows
display(df_timit.sample(5))

,sentence_number,data_group,dialect,gender,speaker,type,text,audio_path,start_sample,end_sample,words_text,words_start_sample,words_end_sample,phonemes_text,phonemes_start_sample,phonemes_end_sample
5243,1754,test,North Midland,M,WJG0,phonetically-diverse,"He said: the crazy fool, half aloud.",test\DR3\MWJG0\SI1754.wav,0,42292,"[he, said, the, crazy, fool, half, aloud]","[2040, 3480, 15791, 17320, 23400, 32040, 35759]","[3480, 11446, 17320, 23400, 29880, 35759, 41420]","[h#, hh, iy, s, eh, dcl, pau, dh, ix, kcl, k, ...","[0, 2040, 2680, 3480, 5800, 8280, 11446, 15791...","[2040, 2680, 3480, 5800, 8280, 11446, 15791, 1..."
2297,344,train,South Midland,M,JSR0,phonetically-compact,I know I didn't meet her early enough.,train\DR4\MJSR0\SX344.wav,0,48845,"[i, know, i, didn't, meet, her, early, enough]","[2372, 4206, 8355, 10120, 15940, 20640, 27708,...","[4206, 8355, 10120, 15940, 20640, 25733, 32793...","[h#, ay, n, ow, ay, dcl, d, ih, dcl, en, m, iy...","[0, 2372, 4206, 5450, 8355, 10120, 11520, 1168...","[2372, 4206, 5450, 8355, 10120, 11520, 11680, ..."
2176,255,train,South Midland,M,GJC0,phonetically-compact,My ideal morning begins with hot coffee.,train\DR4\MGJC0\SX255.wav,0,49357,"[my, ideal, morning, begins, with, hot, coffee]","[2372, 6858, 15164, 22308, 32233, 35108, 40080]","[5880, 15164, 22308, 31201, 35108, 40080, 46633]","[h#, m, ay, q, ay, dcl, d, ih, l, m, ao, r, nx...","[0, 2372, 3520, 5880, 6858, 10109, 11655, 1182...","[2372, 3520, 5880, 6858, 10109, 11655, 11822, ..."
2530,1,train,South Midland,M,STF0,dialect,She had your dark suit in greasy wash water al...,train\DR4\MSTF0\SA1.wav,0,55296,"[she, had, your, dark, suit, in, greasy, wash,...","[2190, 4840, 8560, 10360, 15624, 25200, 27520,...","[4840, 9160, 10360, 15624, 23310, 27520, 33921...","[h#, s, iy, hv, ae, dcl, jh, axr, dcl, d, aa, ...","[0, 2190, 3870, 4840, 5480, 7960, 8560, 9160, ...","[2190, 3870, 4840, 5480, 7960, 8560, 9160, 103..."
627,347,train,Northern,M,BJV0,phonetically-compact,The patient and the surgeon are both recuperat...,train\DR2\MBJV0\SX347.wav,0,57856,"[the, patient, and, the, surgeon, are, both, r...","[1851, 2719, 10253, 12040, 13080, 19640, 21320...","[2719, 10253, 12040, 13080, 19640, 21320, 2596...","[h#, dh, ax, pcl, p, ey, sh, ix, n, tcl, t, eh...","[0, 1851, 2356, 2719, 3640, 4440, 6360, 7690, ...","[1851, 2356, 2719, 3640, 4440, 6360, 7690, 818..."


Now, let's play one sentence from the dataset:

In [5]:
t.sound.play_timit(df_timit.loc[0, :])

## Synthetic speech dataset

The metadata about the dataset containing synthetic speech in different audio settings is loaded using the function `load_synthetic_speech_data` defined in the module `toolbox\data_loading.py`. Once loaded, the data is saved in the project cache as a Pandas dataframe. Let's load it and show a couple of rows:

In [6]:
# Load the meta data about the synthetic speech dataset from the cache.
df_synthetic_speech = pd.read_pickle(paths.cache.df_synthetic_speech)

# Show the top 5 rows.
display(df_synthetic_speech.sample(5))

,speaker,length,variant,segment,fullness,clarity,audio_path
2282,kevin,5s,tv,6,NaN,NaN,cut_5_s\kevin\tele\6.wav
2106,kevin,5s,zoom_augmented,19,1,4,cut_5_s\kevin\1-4\19.wav
2779,kimberly,5s,clean,30,NaN,NaN,cut_5_s\kimberly\orig\30.wav
789,joey,5s,zoom_augmented,11,1,1,cut_5_s\joey\1-1\11.wav
429,joanna,5s,zoom_augmented,31,1,1,cut_5_s\joanna\1-1\31.wav


Let's play one of the first 5s segments from the database:

In [7]:
# Define the properties of the file to play.
mask = (
        (df_synthetic_speech['speaker'] == 'joanna')
        & (df_synthetic_speech['length'] == '5s')
        & (df_synthetic_speech['segment'] == 10)
        & (df_synthetic_speech['variant'] == 'babble')
)

# Show the file data.
display(
    df_synthetic_speech.loc[mask, :]
)

# Play.
t.sound.play_synthetic_speech(df_synthetic_speech.loc[mask, :])

,speaker,length,variant,segment,fullness,clarity,audio_path
559,joanna,5s,babble,10,NaN,NaN,cut_5_s\joanna\babb\10.wav


# Implementation of the DHASP model as the objective function

It is a declared goal of the project to base the objective function of the model on the [Differentiable Hearing Aid Speech Processing (DHASP)](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571) framework. This framework has been developed to optimize the signal processing in a hearing aid so that the signal perceived by a hearing-impaired person would be as close as possibe to that perceived by a person with normal hearing.

<center>
<br>
<img src="resources/graphics/dhasp_original_framework.jpg"
alt="DHASP framework as proposed by its creators"
title="Original DHASP framework"
width="600"/>
<br>
</center>

The original framework compares therefore the output of auditory processing of a signal by a person with normal hearing to the output of the processing of **the same** signal by a person with impaired hearing (figure XXX). We, on the other hand, would like to alter this framework so that it compares the result of auditory processing of two **different** signals - noisy and noise-free - by the same, normal hearing person (Figure XXX). When constructed this way, the framework can tweak the equalization of the noisy signal so that it resembles its noise-free counterpart to a highest possible degree.

<center>
<br>
<img src="resources/graphics/dhasp_proposed_framework.jpg"
alt="DHASP framework as proposed by its creators"
title="Original DHASP framework"
width="600"/>
<br>
</center>

## The EQ

I have chosen that the EQ applied to the noisy speech with the goal of improving its intelligibility will consist of 8 fourth order gammatone bandpass filters, whose:
 - center frequencies are logarithmically spaced between 100 Hz and 8000 Hz
 - bandwidths are set to their center frequencies divided by 2.3
 - peak gain is set to unity, i.e. 0 dB.

Summarising, the coefficients of the $i^{\text{th}}$ filter are given by:

$$
\mathbf{h_{\text{eq}}^{(i)}}[n] =
c_{\text{norm}}^{(i)}
\left(  \frac {n} {f_s}  \right) ^ {(N - 1)}
\exp \left(  -2 \pi b_{\text{eq}}^{(i)} \frac {n} {f_s}  \right)
\cos \left(  2 \pi f_{\text{eq}}  \frac {n} {f_s}   \right)
$$

where:
- $n$ is the sample number
- $f_s$ is the sampling frequency
- $f_{\text{eq}}^{(i)}$ is the center frequency of the $i^{\text{th}}$ filter
- $b_{\text{eq}}^{(i)}$ is the bandwidth of the filter.
- $N$ is the order of the *gammatone* filter, in our case set to 4
- $c_{\text{norm}}^{(i)}$ is a constant normalizing the peak gain of the $i^{\text{th}}$ filter to 0 dB, i.e.:

$$
\begin{align}
c_{\text{norm}}^{(i)}
 &= \left| \sum_n  \mathbf{h_{\text{eq}}^{(i)}}[n] \exp \left(  -2j \pi f_{\text{eq}}^{(i)} \frac {n} {f_s}   \right) \right| \\
 &= \sqrt{
  \left( \sum_n  \mathbf{h_{\text{eq}}^{(i)}}[n] \cos \left( 2 \pi f_{\text{eq}}^{(i)} \frac {n} {f_s}  \right)  \right)^2
   + \left( \sum_n  \mathbf{h_{\text{eq}}^{(i)}}[n] \sin \left( 2 \pi f_{\text{eq}}^{(i)} \frac {n} {f_s}  \right)  \right)^2
   }
\end{align}
$$


The total response of this filterbank, i.e. the sum of the outputs of its filters, is then given by:

$$
\mathbf{h_{\text{eq}}}[n] = \sum_i \mathbf{h_{\text{eq}}^{(i)}}[n]
$$

Having defined the filters, we can assign a gain $g_i$ to each frequency band, apply the modified filter it to a speech signal $\mathbf{x_i}$, and feed the result to our neural network in order to optimize the gains $g_i$ for best speech intelligibility by minimizing the loss function $L$:

$$
G_{\text{optimized}} = \underset{G}{\text{argmin}} L \left( f(G, X), Y \right)
$$

where: $L$ is the loss function which we will define in the following chapters, $ G = \left\{ {g_1, g_2, \cdots, g_8}  \right\}$ a matrix containing the gains for our EQ, $X = \left\{ \mathbf{x_1}, \mathbf{x_2}, \cdots  \right}$  matrix in which each row represents an speech signal, and $Y = \left\{ \mathbf{y_1}, \mathbf{y_2}, \cdots  \right}$ a matrix where each row represents a corresponding target (clear) speech signal. $f(G, \mathbf{x})$ represents a speech signal after having been fitered with our EQ, i.e.:

$$
\begin{align}
f(G, x)
&= \text{conv} \left(  G  \mathbf{h_{\text{eq}}}, \mathbf{x}  \right) \\
&= \text{conv} \left(  \sum_i g_i \mathbf{h_{\text{eq}}^{(i)}}, \mathbf{x}  \right)
\end{align}
$$



## The auditory model

The structure of the auditory model is shown in Figure XXX. The input signal first goes through two filterbanks: an analysis filterbank and a control filterbank, both consisting of 32 bandpass filters with varying properties. Based on the output of the control fliterbank, a dynamic-range compression gain is calculated for frequency band and applied to the outputs of the filters in the analysis filterbank. The output of the auditory model is then calculated as amplitude envelopes (in dB) of the compressed outputs of the filters in the analysis filterbank.

<center>
<br>
<img src="resources/graphics/dhasp_differentiable_auditory_model.jpg"
alt="DHASP framework as proposed by its creators"
title="Original DHASP framework"
width="600"/>
<br>
</center>

### The filterbanks

The coefficients of the filterbanks in the auditory model are calculated in the same way as the coefficients for the EQ applied to the noisy signal described in section XXX, i.e.:

$$
\mathbf{h}[n] =
c_{\text{norm}}^{(i)}
\left(  \frac {n} {f_s}  \right) ^ {(N - 1)}
\exp \left(  -2 \pi b^{(i)} \frac {n} {f}  \right)
\cos \left(  2 \pi f  \frac {n} {f_s}   \right)
$$


#### Center frequencies


In the **analysis** filterbank, the center frequencies are in the Mel scale and cover the e range from 80 Hz to 8 kHz. To calculate them, I have created functions that convert Hz to Mel and vice versa. The computation is then conducted as follows:

In [12]:
# Define the number of filters: I.
I = 32

# Calculate the frequencies for the analysis filterbank.
f_a = t.dsp.mel2hz(torch.linspace(
    t.dsp.hz2mel(80),
    t.dsp.hz2mel(8000),
    I
)).reshape(I, 1)

# Show the first 3 and last 3 frequencies.
print(f'{f_a[:3, :]=}')
print(f'{f_a[-3:, :]=}')

f_a[:3, :]=tensor([[ 80.0000],
        [143.1066],
        [211.3189]], dtype=torch.float64)
f_a[-3:, :]=tensor([[6746.3509],
        [7348.8016],
        [7999.9987]], dtype=torch.float64)



In the **control** filterbank, the center frequencies are given by equation 3 in the [DHASP article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9414571).

$$
f_{\text{c}}^{(i)} =
165.4
\left(
    10^{
        (1 + s)
        \log_{10}
        \left( 1 + f_{\text{a}}^{(i)} / 165.4 \right)
        }
    - 1
\right)
$$

Here, I have corrected a mistake in the paper, where $f_{\text{a}}$ was replaced by $f_{\text{c}}$.

#### Bandwidths

The bandwidths of the analysis filter are in the equivalent rectangular bandwidth (ERB) scale for the normal hearing model. The formula for calculating the ERB for a given frequency is taken from the paper: [Suggested formulae for calculating auditory-filter bandwidths and excitation patterns](https://pubmed.ncbi.nlm.nih.gov/6630731/). I have implemented the formula in the function `erb`, so that the bandwidths can be calculated as:

In [14]:
# Calculate the bandwidths for the analysis filterbank.
b_a = t.dsp.erb(f_a)

# Show the first 3 and last 3 bandwidths.
print(f'{f_a[:3, :]=}')
print(f'{f_a[-3:, :]=}')

f_a[:3, :]=tensor([[ 80.0000],
        [143.1066],
        [211.3189]], dtype=torch.float64)
f_a[-3:, :]=tensor([[6746.3509],
        [7348.8016],
        [7999.9987]], dtype=torch.float64)




[Suggested formulae for calculating auditory-filter bandwidths and excitation patterns](https://pubmed.ncbi.nlm.nih.gov/6630731/)

# End